### Домашняя работа
Для выполнения домашнего задания необходимо взять boston house-prices datase (sklearn.datasets.load_boston) и сделать тоже самое для задачи регрессии (попробовать разные алгоритмы, поподбирать параметры, вывести итоговое качество).

#### Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
#from jupyterthemes import jtplot
#jtplot.style()

#### Загружаем датасет

In [2]:
boston = load_boston()
X = boston["data"]
y = boston["target"]
names = boston["feature_names"]

In [3]:
X = pd.DataFrame(X, columns=names)
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


#### Нормализация

In [4]:
sc = StandardScaler()
X = sc.fit_transform(X)

#### Обучающая и тестовая выборки

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

#### Функция для перебора параметров

In [6]:
def gridsearch(  model, cv, params, model_name, X = X_train, y = y_train):
    grid = GridSearchCV(model, params, scoring = 'r2', cv = cv, n_jobs= -1)
    grid.fit(X, y)
    print('Обучение модели {} завершено'.format(model_name))
    return(grid)

In [7]:
cv = 10

### DecisionTreeRegressor

In [8]:
from sklearn.tree import DecisionTreeRegressor

tr = DecisionTreeRegressor()
params = {'criterion':['mse','friedman_mse'],
          'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150], 
          'min_samples_leaf':[1, 2, 3],
          'min_samples_split':[2,3,4]}
grid_tr = gridsearch(tr, cv, params, 'DecisionTreeRegressor')

Обучение модели DecisionTreeRegressor завершено


### KNeighborsRegressor

In [9]:
from sklearn.neighbors import KNeighborsRegressor

knr = KNeighborsRegressor()
params = {'n_neighbors': range(1,10),
         'p':[1, 2],
         'algorithm': ['auto','ball_tree', 'kd_tree', 'brute']}
grid_knr = gridsearch(knr, cv, params, 'KNeighborsRegressor')

Обучение модели KNeighborsRegressor завершено


### SGDRegressor

In [10]:
from sklearn.linear_model import SGDRegressor

sgd = SGDRegressor()
params = {'alpha' : 10.0**-np.arange(1,7),
          'l1_ratio':[.05, .15, .5, .7, .9, .95, .99, 1]}
grid_sgd = gridsearch(sgd, cv, params, 'SGDRegressor')

Обучение модели SGDRegressor завершено


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


### MLPRegressor

In [11]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor()
params = {"hidden_layer_sizes": [1,50],
          "activation": ["identity", "logistic", "tanh"],
          "solver": ["lbfgs", "sgd", "adam"], 
          "alpha": [0.00005,0.0005]}
grid_mlp= gridsearch(mlp, cv, params, 'MLPRegressor')

Обучение модели MLPRegressor завершено


### RandomForestRegressor

In [12]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
params = {'n_estimators': [500, 700, 1000], 'max_depth': [None, 1, 2, 3], 'min_samples_split': [2, 3]}
grid_rfr= gridsearch(rfr, cv, params, 'RandomForestRegressor')

Обучение модели RandomForestRegressor завершено


In [13]:
estimators = {
    'DecisionTree': grid_tr,
    'KNeighbors': grid_knr,
    'SGD': grid_sgd,
    'MLP': grid_mlp,
    'RandomForest': grid_rfr
}

In [14]:
for i in estimators:
    model = estimators[i]
    print('Модель - {}, CV best score - {}, Validation best score - {}'.format(i, model.best_score_, r2_score(model.best_estimator_.predict(X_test), y_test)))

Модель - DecisionTree, CV best score - 0.7551216539485625, Validation best score - 0.8596696246028717
Модель - KNeighbors, CV best score - 0.7113025969771024, Validation best score - 0.4694334564120832
Модель - SGD, CV best score - 0.6336888613481046, Validation best score - 0.5998984114181487
Модель - MLP, CV best score - 0.7011014733343155, Validation best score - 0.8831891814876986
Модель - RandomForest, CV best score - 0.8292091924877518, Validation best score - 0.8828691098194522
